## Biblioteca Dash com Bootstrap components

### 1. Dashboard Simples com Gráfico Estático

In [ ]:
import webbrowser #inicio
import dash
from dash import html, dcc
import dash_bootstrap_components as dbc
import plotly.express as px

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
fig = px.bar(x=["A", "B", "C"], y=[4, 2, 6], title="Gráfico Estático")
app.layout = dbc.Container([
dbc.Row([
    dbc.Col([
        dcc.Graph(figure=fig)
    ])
])
])

webbrowser.open("http://127.0.0.1:8050") #antes do main
if __name__ == "__main__":
    app.run(debug=True)

### 2. Dashboard com Filtro Interativo

In [ ]:
import webbrowser #inicio
import dash
from dash import html, dcc
import dash_bootstrap_components as dbc
import plotly.express as px
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

data = {
    "Categoria A": [1, 2, 3],
    "Categoria B": [3, 1, 2],
    "Categoria C": [2, 3, 1]
}

opcoes = list(data.keys())
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(id="dropdown", options=[{"label": k, "value": k} for k in
            opcoes], value="Categoria A"),
            dcc.Graph(id="graph")
        ])
    ])
])

@app.callback(
    dash.Output("graph", "figure"),
    dash.Input("dropdown", "value")
)

def update_graph(categoria):
    return px.line(y=data[categoria], title=f"Dados de {categoria}")

webbrowser.open("http://127.0.0.1:8050") #antes do main
if __name__ == "__main__":
    app.run(debug=True)

### 3. Dashboard com Formulário e Armazenamento Local

In [ ]:
import webbrowser #inicio
import dash
from dash import html, dcc
import dash_bootstrap_components as dbc

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    dcc.Store(id="store-data"),
    dbc.Form([
        dbc.Input(id="input-text", placeholder="Introduza algo..."),
        dbc.Button("Guardar", id="save-btn", color="primary", className="mt-2"),
        html.Div(id="output")
        ])
])

@app.callback(
    dash.Output("store-data", "data"),
    dash.Input("save-btn", "n_clicks"),
    dash.State("input-text", "value"),
    prevent_initial_call=True
)

def store_input(n, val):
    return {"value": val}

@app.callback(
    dash.Output("output", "children"),
    dash.Input("store-data", "data")
)

def display_data(data):
    if data:
        return dbc.Alert(f"Guardado: {data['value']}", color="success")
    return ""

webbrowser.open("http://127.0.0.1:8050") #antes do main
if __name__ == "__main__":
    app.run(debug=True)

### 4. Dashboard Multi-Página com Navbar


In [ ]:
import webbrowser #inicio
import dash # Importa a biblioteca principal do Dash
from dash import html, dcc # Importa os componentes HTML e Dash Core
import dash_bootstrap_components as dbc # Importa os componentes Bootstrap para Dash

# Inicializa a aplicação Dash com tema Bootstrap
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define o layout da aplicação
app.layout = dbc.Container([ # Container principal para responsividade
    html.H2("Formulário com Armazenamento Temporário", className="my-4"), # Título da aplicação
    dbc.Row([ # Linha de layout Bootstrap
        dbc.Col([ # Coluna da linha
            dbc.Form([ # Formulário Bootstrap
                html.Div([ # Div que agrupa o label e input
                    dbc.Label("Nome:", html_for="input-text"), # Etiqueta do campo de input
                    dbc.Input(id="input-text", placeholder="Introduza o seu nome", type="text") # Campo de entrada de texto
                ]),
            
            dbc.Button("Guardar", id="save-btn", color="primary", className="mt-3") # Botão de submissão
            ]),
        html.Hr(), # Linha horizontal de separação
        html.Div(id="output", className="mt-3") # Div onde será exibida a mensagem após guardar
        
        ], md=6) # Define que a coluna ocupa 6 das 12 colunas do grid Bootstrap
    ]),

    dcc.Store(id="store-data") # Componente para guardar dados temporários no browser
    
], fluid=True) # Define que o container é fluido (100% de largura)

# Callback para guardar os dados introduzidos no input
@app.callback(
    dash.Output("store-data", "data"), # Define que os dados serão guardados no dcc.Store
    dash.Input("save-btn", "n_clicks"), # Ação de clique no botão
    dash.State("input-text", "value"), # Lê o valor atual do input
    prevent_initial_call=True # Impede que o callback seja chamado no carregamento da página
)

def guardar_dados(n_clicks, valor):
    if valor and valor.strip(): # Verifica se o valor não está vazio
        return {"nome": valor} # Retorna um dicionário com o nome
    return dash.no_update # Se o valor estiver vazio, não atualiza nada

# Callback para mostrar os dados guardados
@app.callback(
    dash.Output("output", "children"), # Define que será atualizada a div de output
    dash.Input("store-data", "data") # O input é o conteúdo armazenado no dcc.Store
)

def mostrar_dados(data):
    if data: # Se existirem dados
        return dbc.Alert(f"Olá, {data['nome']}! Os teus dados foram guardados.", color="success") # Mensagem de sucesso
    return "" # Se não houver dados, retorna vazio

# Executa o servidor da aplicação
webbrowser.open("http://127.0.0.1:8050") #antes do main
if __name__ == "__main__":
    app.run(debug=True)

### 5. Dashboard com Upload de Ficheiro e Análise de Dados

In [ ]:
import webbrowser #inicio
import dash
from dash import html, dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import base64
import io

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dcc.Upload(id="upload-data", children=html.Div(["Arraste ou selecione um ficheiro CSV"]), style={"padding": "20px", "border": "2px dashed #ccc"}),
    dbc.Spinner(children=[dcc.Graph(id="upload-graph")], size="lg")
])

@app.callback(dash.Output("upload-graph", "figure"),
              dash.Input("upload-data", "contents")
)

def process_file(contents):
    if contents:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        fig = px.histogram(df, x=df.columns[1])
        return fig
    
    return px.histogram(title="Nenhum ficheiro carregado")

webbrowser.open("http://127.0.0.1:8050") #antes do main
if __name__ == "__main__":
    app.run(debug=True)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[18], line 29, in process_file(contents=None)
     26     fig = px.histogram(df, x=df.columns[1])
     27     return fig
---> 29 return px.histogram(title="Nenhum ficheiro carregado")
        px = <module 'plotly.express' from 'c:\\Users\\HP\\anaconda3\\envs\\Podinis\\lib\\site-packages\\plotly\\express\\__init__.py'>

File c:\Users\HP\anaconda3\envs\Podinis\lib\site-packages\plotly\express\_chart_types.py:490, in histogram(
    data_frame=None,
    x=None,
    y=None,
    color=None,
    pattern_shape=None,
    facet_row=None,
    facet_col=None,
    facet_col_wrap=0,
    facet_row_spacing=None,
    facet_col_spacing=None,
    hover_name=None,
    hover_data=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    patte